## Segmenting and Clustering Neighborhoods in Toronto

Week 3 assignment for *pkali*, part #3

In [11]:
import pandas as pd
import numpy as np

In [12]:
from bs4 import BeautifulSoup
import requests
base = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
soup = BeautifulSoup(requests.get(base).text, 'lxml')
tabs = soup.find("table",{"class":"wikitable sortable"})
df = pd.read_html(str(tabs))[0] #read only the first tab
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)
df.reset_index(drop=True,inplace=True)

In [13]:
geospatial = pd.read_csv('http://cocl.us/Geospatial_data')
df = df.merge(geospatial, left_on='Postal Code', right_on='Postal Code')

#### Get Toronto Boroughs only 

In [22]:
df_t = df[df.Borough.str.contains('Toronto')]

In [33]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="pkali_notebook")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_t['Latitude'], 
                                           df_t['Longitude'], 
                                           df_t['Borough'], 
                                           df_t['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

#### Map of Toronto with marked neigborhoods

In [35]:
map_toronto

#### Define Foursquare Credentials and Version

In [36]:
CLIENT_ID = 'W4Z5L01JHWF0ARMJNDDVE5KOBVNQDVQON1RN15MXW1HOTVYP' # your Foursquare ID
CLIENT_SECRET = 'MKRMHSSBSBESO33A2EJDA0ZORVBGC4AHVYO4IAO1JU0XGVXG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 10
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [40]:
toronto_venues = getNearbyVenues(names=df_t['Neighborhood'],
                                   latitudes=df_t['Latitude'],
                                   longitudes=df_t['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town,

In [44]:
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
...,...,...,...,...,...,...,...
347,Business reply mail Processing Centre,43.662744,-79.321558,Chick-n-Joy,43.665181,-79.321403,Fast Food Restaurant
348,Business reply mail Processing Centre,43.662744,-79.321558,The Green Wood,43.664728,-79.324117,Restaurant
349,Business reply mail Processing Centre,43.662744,-79.321558,Ashbridges Bay Skatepark,43.662548,-79.315631,Skate Park
350,Business reply mail Processing Centre,43.662744,-79.321558,East End Garden Centre & Hardware,43.664564,-79.324471,Garden Center


In [56]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

#### Find most common venues in each neigborhood

In [57]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [66]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

#neighborhoods_venues_sorted.head()

### Cluster neigborhoods
Run *k*-means to cluster the neighborhood into 5 clusters, based on New York example lab.

In [67]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_t

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Breakfast Spot,Coffee Shop,Historic Site,Park,Distribution Center,Bakery,Restaurant,Spa,Gym / Fitness Center,Dance Studio
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4,Coffee Shop,Arts & Crafts Store,Park,Creperie,Mexican Restaurant,Distribution Center,Italian Restaurant,Beer Bar,Food & Drink Shop,Food Truck
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Café,Comic Shop,Clothing Store,Music Venue,Theater,Thai Restaurant,Tea Room,Plaza,Burrito Place,Dance Studio
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,4,Coffee Shop,Food Truck,Gastropub,Cosmetics Shop,Creperie,Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Gym,Fountain
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Trail,Pub,Asian Restaurant,Health Food Store,Wine Bar,Department Store,Dog Run,Distribution Center,Diner,Dessert Shop


### Cluster visualization

In [70]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'],
                                  toronto_merged['Longitude'],
                                  toronto_merged['Neighborhood'],
                                  toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster examination
 Examine and name all clusters.

#### Cluster #1

In [71]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,East Toronto,0,Greek Restaurant,Ice Cream Shop,Yoga Studio,Cosmetics Shop,Fruit & Vegetable Store,Italian Restaurant,Brewery,Diner,Dog Run,Distribution Center
62,Central Toronto,0,Garden,Ice Cream Shop,Home Service,Wine Bar,Dessert Shop,Donut Shop,Dog Run,Distribution Center,Diner,Department Store


#### Cluster #1 name: Sweet Mediterranean Tooth

----
#### Cluster #2

In [75]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,1,Breakfast Spot,Coffee Shop,Historic Site,Park,Distribution Center,Bakery,Restaurant,Spa,Gym / Fitness Center,Dance Studio
20,Downtown Toronto,1,Cocktail Bar,Vegetarian / Vegan Restaurant,Park,Coffee Shop,Concert Hall,Fountain,Liquor Store,Museum,Farmers Market,Restaurant
30,Downtown Toronto,1,Speakeasy,Plaza,Hotel,Steakhouse,Restaurant,Gym / Fitness Center,Café,Concert Hall,Vegetarian / Vegan Restaurant,Cosmetics Shop
36,Downtown Toronto,1,Skating Rink,Supermarket,Park,Performing Arts Venue,Sporting Goods Shop,Hotel,Plaza,Salad Place,Lake,Dessert Shop
37,West Toronto,1,Wine Bar,Asian Restaurant,Ice Cream Shop,Korean Restaurant,Cuban Restaurant,New American Restaurant,Cocktail Bar,Brewery,Beer Store,Pizza Place
47,East Toronto,1,Pet Store,Sushi Restaurant,Liquor Store,Brewery,Italian Restaurant,Ice Cream Shop,Fast Food Restaurant,Fish & Chips Shop,Park,Gym
67,Central Toronto,1,Park,Gym / Fitness Center,Breakfast Spot,Sandwich Place,Hotel,Food & Drink Shop,Department Store,Dance Studio,Donut Shop,Cuban Restaurant
69,West Toronto,1,Speakeasy,Italian Restaurant,Furniture / Home Store,Gastropub,Music Venue,Park,Café,Arts & Crafts Store,Bar,Flea Market
73,Central Toronto,1,Yoga Studio,Spa,Diner,Dessert Shop,Mexican Restaurant,Coffee Shop,Clothing Store,Restaurant,Salon / Barbershop,Chinese Restaurant
74,Central Toronto,1,Indian Restaurant,American Restaurant,Coffee Shop,Café,Middle Eastern Restaurant,Burger Joint,Donut Shop,BBQ Joint,Park,Vegetarian / Vegan Restaurant


#### Cluster #2 name: Fast, fast world!

----
#### Cluster #3

In [77]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Downtown Toronto,2,Café,Comic Shop,Clothing Store,Music Venue,Theater,Thai Restaurant,Tea Room,Plaza,Burrito Place,Dance Studio
19,East Toronto,2,Trail,Pub,Asian Restaurant,Health Food Store,Wine Bar,Department Store,Dog Run,Distribution Center,Diner,Dessert Shop
25,Downtown Toronto,2,Café,Grocery Store,Coffee Shop,Italian Restaurant,Diner,Candy Store,Restaurant,Department Store,Distribution Center,Dessert Shop
31,West Toronto,2,Bakery,Supermarket,Music Venue,Brewery,Bar,Grocery Store,Bank,Middle Eastern Restaurant,Café,Wine Bar
42,Downtown Toronto,2,Café,Beer Bar,Gym / Fitness Center,Pub,Restaurant,Bakery,Tea Room,Coffee Shop,Gym,Distribution Center
43,West Toronto,2,Coffee Shop,Bakery,Pet Store,Café,Restaurant,Bar,Italian Restaurant,Gym,Furniture / Home Store,Fountain
48,Downtown Toronto,2,Café,Coffee Shop,Gym,Bakery,Restaurant,Pub,Tea Room,Gym / Fitness Center,Museum,Creperie
54,East Toronto,2,Gay Bar,Fish Market,Pet Store,Sandwich Place,Bookstore,Italian Restaurant,Ice Cream Shop,Bakery,Coffee Shop,Food
75,West Toronto,2,Gift Shop,Movie Theater,Restaurant,Coffee Shop,Italian Restaurant,Dog Run,Dessert Shop,Eastern European Restaurant,Cuban Restaurant,Dance Studio
79,Central Toronto,2,Italian Restaurant,Dessert Shop,Sushi Restaurant,Café,Seafood Restaurant,Indian Restaurant,Pizza Place,Coffee Shop,Food,Comic Shop


#### Cluster #3 name: Caffè Italiano

----
#### Cluster #4

In [81]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Central Toronto,3,Park,Bus Line,Swim School,Wine Bar,Eastern European Restaurant,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio
68,Central Toronto,3,Park,Bus Line,Sushi Restaurant,Jewelry Store,Trail,Concert Hall,Cosmetics Shop,Creperie,Donut Shop,Dance Studio
91,Downtown Toronto,3,Park,Trail,Playground,Wine Bar,Donut Shop,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant


#### Cluster #4 name: Walk the Dog 

----
#### Cluster #5

In [82]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,4,Coffee Shop,Arts & Crafts Store,Park,Creperie,Mexican Restaurant,Distribution Center,Italian Restaurant,Beer Bar,Food & Drink Shop,Food Truck
15,Downtown Toronto,4,Coffee Shop,Food Truck,Gastropub,Cosmetics Shop,Creperie,Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Gym,Fountain
24,Downtown Toronto,4,Coffee Shop,Spa,Gastropub,Modern European Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Japanese Restaurant,Food Truck,Food & Drink Shop,Garden


#### Cluster #5 name: Hipster's Paradise

In [84]:
### Map with cluster names
cluster_names = ['Sweet Mediterranean Tooth',
                 'Fast, fast world!',
                 'Caffe Italiano',
                 'Walk the Dog',
                 "Hipster's Paradise"]
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'],
                                  toronto_merged['Longitude'],
                                  toronto_merged['Neighborhood'],
                                  toronto_merged['Cluster Labels']):
    label = folium.Popup( cluster_names[cluster]+': '+str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters